In [1]:
import pandas as pd

In [2]:
!pwd

/Users/pranav/workspace/UIUC/Fall2020/CS_PSL/Project_3


In [3]:
split1 = pd.read_csv('split_1/train.tsv', sep='\t')

In [4]:
split1.head()

,id,sentiment,review
0,1,1,Naturally in a film who's main themes are of m...
1,4,0,Afraid of the Dark left me with the impression...
2,7,0,This has to be one of the biggest misfires eve...
3,8,0,"This is one of those movies I watched, and won..."
4,17,0,This movie was dreadful. Biblically very inacc...


In [5]:
split1['sentiment'] = split1['sentiment'].astype('int')

In [6]:
split1.dtypes

id            int64
sentiment     int64
review       object
dtype: object

In [7]:
## Review cleanup (this process takes a significant amount of time)
# Removing stop words, @ mentions, webpages and special characters

from nltk.corpus import stopwords # nltk.download('stopwords') before importing
import re

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def clean(review):
    
    stage1 = [word for word in review.lower().split() if word not in stopwords.words('english')] # stopword removal
    stage2 = [REPLACE_NO_SPACE.sub("", line.lower()) for line in stage1]
    stage3 = [REPLACE_WITH_SPACE.sub(" ", line) for line in stage2]
    return ' '.join(stage3)

In [8]:
%%time

split1['review_cleaned'] = split1['review'].apply(clean)

CPU times: user 7min 45s, sys: 2min 6s, total: 9min 51s
Wall time: 9min 54s


In [9]:
train_1x = split1['review_cleaned']
train_1y = split1['sentiment']

In [10]:
## Analysis

from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
vector = TfidfVectorizer(max_features = 1000, ngram_range = (1, 2), stop_words = 'english')
%time vector.fit(train_1x)

CPU times: user 10.9 s, sys: 219 ms, total: 11.1 s
Wall time: 11.2 s


TfidfVectorizer(max_features=1000, ngram_range=(1, 2), stop_words='english')

In [147]:
vector = TfidfVectorizer(vocabulary = vocab, ngram_range = (1, 2), stop_words = 'english')
%time vector.fit(train_1x)

Wall time: 4.71 s


TfidfVectorizer(ngram_range=(1, 2), stop_words='english',
                vocabulary=dict_keys(['naturally', 'film', 'main', 'themes', 'loss', 'surprising', 'rated', 'highly', 'older', 'viewers', 'younger', 'ones', 'enjoy', 'pace', 'constant', 'characters', 'engaging', 'relationships', 'natural', 'showing', 'need', 'tears', 'emotion', 'fear', 'violence', 'short', 'story', 'ready', 'perfect', 'sm..., 'joan', 'spanish', 'joseph', 'directing', 'gang', 'anthony', 'context', 'speech', 'dollars', 'br overall', 'heads', 'felt like', 'twist', 'priest', 'fault', 'sucks', 'batman', 'parody', 'featuring', 'eat', 'test', 'andy', 'thoughts', 'stone', 'danny', 'genuinely', 'pleasure', 'grand', 'halloween', 'author', 'allen', 'river', 'howard', 'corny', 'nonsense', 'alex', 'surely', 'jackson']))

In [43]:
train_1x_transformed = vector.transform(train_1x)

In [156]:
train_1x_transformed

<25000x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 1427174 stored elements in Compressed Sparse Row format>

In [109]:
from sklearn.naive_bayes import MultinomialNB

In [13]:
from sklearn.linear_model import LogisticRegression

In [52]:
from sklearn.ensemble import RandomForestClassifier

In [136]:
from xgboost import XGBClassifier

In [135]:
import sys
!{sys.executable} -m pip install xgboost

In [157]:
xgb = XGBClassifier()

In [112]:
nb = MultinomialNB()

In [53]:
rf = RandomForestClassifier(n_estimators = 1000, verbose = 2)

In [54]:
rf.fit(train_1x_transformed, train_1y)

RandomForestClassifier(n_estimators=1000)

In [59]:
from sklearn.neural_network import MLPClassifier

NN = MLPClassifier(verbose=2)
NN.fit(train_1x_transformed, train_1y)

Iteration 1, loss = 0.53025972
Iteration 2, loss = 0.34571696
Iteration 3, loss = 0.31803466
Iteration 4, loss = 0.31125274
Iteration 5, loss = 0.30764142
Iteration 6, loss = 0.30563182
Iteration 7, loss = 0.30391523
Iteration 8, loss = 0.30264072
Iteration 9, loss = 0.30127874
Iteration 10, loss = 0.29993344
Iteration 11, loss = 0.29800184
Iteration 12, loss = 0.29663987
Iteration 13, loss = 0.29442219
Iteration 14, loss = 0.29246888
Iteration 15, loss = 0.29066464
Iteration 16, loss = 0.28807279
Iteration 17, loss = 0.28532660
Iteration 18, loss = 0.28328976
Iteration 19, loss = 0.28058753
Iteration 20, loss = 0.27724138
Iteration 21, loss = 0.27360825
Iteration 22, loss = 0.27096189
Iteration 23, loss = 0.26694364
Iteration 24, loss = 0.26264752
Iteration 25, loss = 0.25877983
Iteration 26, loss = 0.25445277
Iteration 27, loss = 0.24981203
Iteration 28, loss = 0.24506888
Iteration 29, loss = 0.24006425
Iteration 30, loss = 0.23429868
Iteration 31, loss = 0.22869109
Iteration 32, los

MLPClassifier(verbose=2)

In [44]:
lg = LogisticRegression(max_iter = 1000)

In [45]:
lg.fit(train_1x_transformed, train_1y)

LogisticRegression(max_iter=1000)

In [158]:
xgb.fit(train_1x_transformed, train_1y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [115]:
nb.fit(train_1x_transformed, train_1y)

MultinomialNB()

In [116]:
nb.predict(train_1x_transformed[0])

array([1])

In [117]:
nb.predict_proba(train_1x_transformed[0])

array([[0.08701922, 0.91298078]])

In [118]:
nb.predict_log_proba(train_1x_transformed[0])

array([[-2.44162628, -0.09104045]])

In [119]:
len(vector.vocabulary_)

2000

In [144]:
vocab = vector.vocabulary_.keys()

In [16]:
### Test it

split1_test_x = pd.read_csv('split_1/test.tsv', sep = '\t')
split1_test_y = pd.read_csv('split_1/test_y.tsv', sep = '\t')

In [17]:
split1_test_x['review_cleaned'] = split1_test_x['review'].apply(clean)

In [46]:
split_id = split1_test_x['id'].to_numpy()

In [47]:
split_id

array([  598, 12048, 40908, ...,  3620, 24858, 13068])

In [55]:
test_1x = vector.transform(split1_test_x['review_cleaned'])

In [56]:
test_1y = split1_test_y['sentiment']

In [57]:
rf_pred = rf.predict_proba(test_1x)

In [60]:
nn_pred = NN.predict_proba(test_1x)

In [127]:
pred1 = nb.predict(test_1x)

In [50]:
pred1_prob = lg.predict_proba(test_1x)

In [159]:
pred1_prob = xgb.predict_proba(test_1x)

In [129]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [130]:
print('Accuracy - {}'.format(accuracy_score(test_1y, pred1)))
print('Confusion matrix - {}'.format(confusion_matrix(test_1y, pred1)))

Accuracy - 0.84548
Confusion matrix - [[10374  2153]
 [ 1710 10763]]


In [131]:
len(pred1_prob)

25000

In [132]:
len(split_id)

25000

In [51]:
file = open("mysubmission.txt","w")
file.write('"id"    "prob"')
file.write("\n")
num = len(split_id)
for x,y in enumerate(split_id):
    string = str(y) + "    "+ str(pred1_prob[x][1])
    file.write(string)
    if x != num - 1:
        file.write("\n")
file.close()
    

In [58]:
file = open("mysubmission_rf.txt","w")
file.write('"id"    "prob"')
file.write("\n")
num = len(split_id)
for x,y in enumerate(split_id):
    string = str(y) + "    "+ str(rf_pred[x][1])
    file.write(string)
    if x != num - 1:
        file.write("\n")
file.close()
    

In [61]:
file = open("mysubmission_nn.txt","w")
file.write('"id"    "prob"')
file.write("\n")
num = len(split_id)
for x,y in enumerate(split_id):
    string = str(y) + "    "+ str(nn_pred[x][1])
    file.write(string)
    if x != num - 1:
        file.write("\n")
file.close()